# NP-Hard Problems

Used [NetworkX](https://networkx.github.io/) to manipulate graphs and [PuLP](http://pythonhosted.org/PuLP/) to solve linear and integer programs.

In [1]:
import networkx as nx
import pulp

Used two graph datasets, both available in [GML](https://en.wikipedia.org/wiki/Graph_Modelling_Language) format. The first is a well-known graph of the social network of a karate club [1]. The second is a network representing the western states power grid in the US [2].

In [2]:
karate = nx.read_gml('karate.gml')
power = nx.read_gml('power.gml')

[1] W. Zachary, An information flow model for conflict and fission in small groups, Journal of Anthropological Research 33, 452-473 (1977). 

[2] D. J. Watts and S. H. Strogatz, Collective dynamics of 'small world' networks, Nature 393, 440-442 (1998). 

## Independent Set

Maximize   :  $\sum_{i=0}^n x_i$

Subject to :  $x_i + x_j \leq 1$ for all $(i,j) \in E$ and $x_i \in$ {0,1} for all i $\in N$



### Function that solves the integer program given a graph as input.



In [3]:
def independent_set_ip(graph):
    """Computes a maximum independent set of a graph using an integer program.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, int)]) The IP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    
    # Create the 'prob' variable to contain the problem data
    prob = pulp.LpProblem("IndependentSet_IP", pulp.LpMaximize)
    
    # Dictionary of node variables(xi's)
    x = pulp.LpVariable.dicts("Node_Variables", graph.nodes(), 0, 1, pulp.LpInteger)
    
    # The objective function is added to 'prob' first
    prob += pulp.lpSum(x[i] for i in graph.nodes()), "Total number of nodes included in the Independent Set"
    
    # The constraints are added to 'prob'
    for (i,j) in graph.edges():
        prob += (x[i] + x[j]) <= 1
        
    # Solve the IP
    prob.solve()
    
    x_optIP_values = list()
    for n in graph.nodes():
        x_optIP_values.append((int(n), int(x[n].varValue)))
    
    return x_optIP_values
    
    pass

The following code outputs the size of the sets computed by your function.

In [4]:
def set_weight(solution):
    """Computes the total weight of the solution of an LP or IP for independent set.
    
    Args:
      - solution (list[int, float]): the LP or IP solution
    
    Returns:
      (float) Total weight of the solution
    
    """
    return sum(value for (node, value) in solution)

In [5]:
karate_ind_set = independent_set_ip(karate)
print "Size of karate set = ", set_weight(karate_ind_set)
power_ind_set = independent_set_ip(power)
print "Size of power set = ", set_weight(power_ind_set)

Size of karate set =  20
Size of power set =  2738


### *Linear programming relaxation* of the integer program. In the integer program, replace each constraint $x_i \in \{0,1\}$ with $0 \leq x_i \leq 1$.



In [6]:
def independent_set_lp(graph):
    """Computes the solution to the linear programming relaxation for the
    maximum independent set problem.
    
    Args:
      - graph (nx.Graph): an undirected graph
    
    Returns:
        (list[(int, float)]) The LP solution as a list of node-value pairs.
        
    """    
    # TODO: implement function
    
    # Create the 'prob' variable to contain the problem data
    prob = pulp.LpProblem("IndependentSet_IP", pulp.LpMaximize)
    
    # Dictionary of node variables(xi's)
    x = pulp.LpVariable.dicts("Node_Variables", graph.nodes(), 0, 1, pulp.LpContinuous)
    
    # The objective function is added to 'prob' first
    prob += pulp.lpSum(x[i] for i in graph.nodes()), "Total number of nodes included in the Independent Set"
    
    # The constraints are added to 'prob'
    for (i,j) in graph.edges():
        prob += (x[i] + x[j]) <= 1
        
    # Solve the IP
    prob.solve()
    
    x_optLP_values = list()
    for n in graph.nodes():
        x_optLP_values.append((int(n), float(x[n].varValue)))
    
    return x_optLP_values
    pass

Let's see how the LP solutions compare to those of the IP.

In [7]:
karate_ind_set_relax = independent_set_lp(karate)
print "Value of karate set = ", set_weight(karate_ind_set_relax)
power_ind_set_relax = independent_set_lp(power)
print "Value of power set = ", set_weight(power_ind_set_relax)

Value of karate set =  20.5
Value of power set =  2758.0


### Implementing a rounding procedure.

A heuristic way to convert a fractional solution to an independent set is as follows. For each node $i$, include the node in the set if $x_i > 1/2$, and discard it otherwise. This will yield a set of $a$ nodes which have $b$ edges between them. By removing at most one node for each edge, this yields an independent set of size at least $a - b$.

In [8]:
def round_solution(solution, graph):
    """Finds the subgraph corresponding to the rounding of
    a solution to the independent set LP relaxation.
    
    Args:
      - solution (list[(int, float)]): LP solution
      - graph (nx.Graph): the original graph
      
    Returns:
        (nx.Graph) The subgraph corresponding to rounded solution
    
    """
    # TODO: implement function
    
    subgraph = graph.copy()
    
    # Discard a node if xi <= 1/2
    for (i,x) in solution:
        if x <= 0.5:
            subgraph.remove_node(i)
            
    return subgraph
    pass

The following function assesses the quality of the heuristic approach.

In [9]:
def solution_quality(rounded, optimal):
    """Computes the percent optimality of the rounded solution.
    
    Args:
      - rounded (nx.Graph): the graph obtained from rounded LP solution
      - optimal: size of maximum independent set
    
    """
    num_nodes = rounded.number_of_nodes() - rounded.number_of_edges()
    return float(num_nodes) / optimal

Let's check the quality of this approach compared to the optimal IP solutions.

In [10]:
karate_rounded = round_solution(karate_ind_set_relax, karate)
karate_quality = solution_quality(karate_rounded, set_weight(karate_ind_set))
print "Quality of karate rounded solution = {:.0f}%".format(karate_quality*100)

power_rounded = round_solution(power_ind_set_relax, power)
power_quality = solution_quality(power_rounded, set_weight(power_ind_set))
print "Quality of power rounded solution = {:.0f}%".format(power_quality*100)

Quality of karate rounded solution = 90%
Quality of power rounded solution = 95%
